In [1]:
# CONSTANTS
salary_cap = 94143000

In [2]:
import urllib.request
from nba_py import player, team
from nba_py.constants import *
from bs4 import BeautifulSoup
from __future__ import print_function
import pandas as pd
import numpy as np
import re
import itertools
import datetime as dtime

In [3]:
def strip_name(name):
    return re.sub(r'[^\w\s]','',name.lower())

In [4]:
def create_salary_csv():
    salaryDT = pd.read_csv('data/contracts_players_contracts.csv')
    salaryDT['formatted_name'] = salaryDT.apply(lambda row: strip_name(row['Player']), axis=1)
    return salaryDT

In [19]:
def find_valid_team_players(team_name, valid_players):
    teamDT = team.TeamCommonRoster(team_id=TEAMS[team_name]['id'],season='2016-17').roster()
    roster = list(teamDT['PLAYER'])
    rookies = 0
    a = list(roster)
    
    for player in a:
        if(len(salaryDT[salaryDT['formatted_name'] == strip_name(player)]) != 0):
            if(salaryDT[salaryDT['formatted_name'] == strip_name(player)]['Signed Using'].iloc[0] == '1st Round Pick'):
                rookies = rookies + 1
                roster.remove(player)
        else:
            roster.remove(player)
        
    for _ in range(5-rookies):
        team_contracts = salaryDT[salaryDT.Player.isin(roster)][['Player','2016-17']]
        team_contracts['2016-17'] = team_contracts['2016-17'].apply(lambda x: float(strip_name('2016-17')))
        team_contracts = team_contracts.sort_values('2016-17', ascending=False)
        roster.remove(team_contracts.iloc[0]['Player'])
    
    valid_players.append(roster)
        
    return valid_players

In [20]:
def find_valid_players():
    valid_players = []
    for team_name in list(TEAMS.keys()):
        valid_players = find_valid_team_players(team_name, valid_players)
    valid_players = list(itertools.chain.from_iterable(valid_players))
    
    return valid_players

In [21]:
def create_master_player_DT():
    teamDTList = []
    for team_name in list(TEAMS.keys()):
        teamDT = team.TeamCommonRoster(team_id=TEAMS[team_name]['id'],season='2016-17').roster()
        teamDTList.append(teamDT)
    return pd.concat(teamDTList)

In [22]:
def get_data(url_string):
    soup = BeautifulSoup(urllib.request.urlopen(url_string).read(), 'lxml')
    vorp = 0
    try:
        result_row = soup.find(id = 'advanced.2016').find_all('td')
        vorp_element = result_row[len(result_row)-1].get_text()
        vorp = float(vorp_element)
    except:
        pass
    return vorp

In [23]:
def find_player_vorp(player_name):
    base_url = 'http://www.basketball-reference.com/players/'
    name_array = strip_name(player_name).split(' ')
    first_name = name_array[0]
    last_name = name_array[len(name_array)-1]
    url_string = base_url + last_name[0] + '/' + last_name[0:5] + first_name[0:2] + '01.html'
    vorp = get_data(url_string)
    return vorp

In [24]:
def create_vorp_array(player_names):
    vorp_array = np.zeros(len(player_names))
    for index, player_name in enumerate(player_names):
        vorp_array[index] = find_player_vorp(player_name)
    return vorp_array

In [25]:
# CREATE masterPlayerDT, filter out any rookies/invalid players to choose
salaryDT = create_salary_csv()

valid_players = find_valid_players()

masterPlayerDT = create_master_player_DT()
masterPlayerDT = masterPlayerDT[masterPlayerDT['PLAYER'] != "LeBron James"]
masterPlayerDT = masterPlayerDT[masterPlayerDT['EXP'] != "R"]
masterPlayerDT = masterPlayerDT.drop(['SEASON', 'TeamID', 'LeagueID', 'NUM', 'BIRTH_DATE', 'SCHOOL', 'PLAYER_ID'], axis=1)
masterPlayerDT = masterPlayerDT[masterPlayerDT.PLAYER.isin(valid_players)]
masterPlayerDT['Salary'] = masterPlayerDT['PLAYER'].apply(lambda x: list(salaryDT[salaryDT['formatted_name'] == strip_name(x)]['2016-17'])[0])

In [26]:
masterPlayerDT['VORP'] = pd.Series(create_vorp_array(masterPlayerDT['PLAYER']), index=masterPlayerDT.index)

In [69]:
masterPlayerDT['VORP/$$$'] = masterPlayerDT.apply(lambda row: float(row['VORP'])/float(strip_name(row['Salary']))*1000000, axis=1)
VORPsortedDT = masterPlayerDT.sort_values('VORP', ascending=False)
valueSortedDT = masterPlayerDT.sort_values('VORP/$$$', ascending=False)

In [73]:
goodContractsDT = pd.merge(VORPsortedDT.head(50), valueSortedDT.head(25), on='PLAYER', how='inner', suffixes=('', '_y'))
goodContractsDT = goodContractsDT.sort_values('VORP', ascending = False)
goodContractsDT = goodContractsDT.drop(['POSITION_y', 'HEIGHT_y', 'WEIGHT_y', 'AGE_y', 'EXP_y', 'Salary_y', 'VORP_y', 'VORP/$$$_y'], axis=1)
goodContractsDT

,PLAYER,POSITION,HEIGHT,WEIGHT,AGE,EXP,Salary,VORP,VORP/$$$
0,Stephen Curry,G,6-3,190,28.0,7,"$12,112,359",9.8,0.809091
1,Russell Westbrook,G,6-3,200,27.0,8,"$26,540,100",8.3,0.312734
2,Kyle Lowry,G,6-0,205,30.0,10,"$12,000,000",6.3,0.525000
3,Patty Mills,G,6-0,185,28.0,7,"$3,578,948",4.9,1.369117
4,Nikola Jokic,C,6-10,250,21.0,1,"$1,358,500",3.0,2.208318
5,Jae Crowder,F,6-6,235,26.0,4,"$6,286,408",2.8,0.445405
6,Trevor Ariza,F,6-8,215,31.0,12,"$7,806,971",2.5,0.320227
7,Ed Davis,F-C,6-10,240,27.0,6,"$6,666,667",2.3,0.345000
8,David West,F,6-9,250,35.0,13,"$1,551,659",1.9,1.224496
9,PJ Tucker,F,6-6,245,31.0,5,"$5,300,000",1.7,0.320755
